In [1]:
from langchain.vectorstores import FAISS
from langchain.memory import VectorStoreRetrieverMemory
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

c:\Users\pc\Documents\Coding\Python\Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [4]:
llm = HuggingFaceEndpoint(
    model="meta-llama/Llama-3.3-70B-Instruct",
    task="text-generation",
    temperature=0
)

model = ChatHuggingFace(llm=llm)

In [5]:
memory_store = FAISS.from_texts([""], embeddings)

In [13]:
from langchain.retrievers import MultiQueryRetriever, ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# Suppose you have your base vectorstore retriever
retriever = memory_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 20, "fetch_k": 50, "lambda_mult": 0.5}
)

In [14]:
memory = VectorStoreRetrieverMemory(retriever=retriever)

In [15]:
def get_history(query: str):
    history_docs = memory.load_memory_variables({'input':query})
    return history_docs.get("history", "")

In [16]:
prompt_template = """
You are a concise and knowledgeable AI assistant.
Use the past conversation if it helps answer the query, otherwise ignore it.

Conversation history:
{history}

User Query:
{query}

Your answer:
"""

In [17]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["history", "query"]
)

prompt

PromptTemplate(input_variables=['history', 'query'], input_types={}, partial_variables={}, template='\nYou are a concise and knowledgeable AI assistant.\nUse the past conversation if it helps answer the query, otherwise ignore it.\n\nConversation history:\n{history}\n\nUser Query:\n{query}\n\nYour answer:\n')

In [18]:
parser = StrOutputParser()

In [19]:
parallel_chain = RunnableParallel({
    'history': RunnableLambda(get_history),
    'query': RunnablePassthrough()
})

In [20]:
main_chain = parallel_chain | prompt | model | parser

In [21]:
def ask(query: str):
    answer = main_chain.invoke(query)
    memory.save_context({"input": query}, {"output": answer})
    return answer

In [22]:
query1 = "Explain the concept of diffusion tarnsformers."
query2 = "Compare its architecture from the standard one."
query3 = "Explain it in more detail."
query4 = "Generate a detailed report on AI revolution in India."
query5 = "Let's continue the previous conversation we were having."

In [23]:
hist_docs = memory_store.similarity_search("", k=100)
len(hist_docs)

1

In [24]:
ask(query1)

'Diffusion transformers are a type of deep learning model that combines the strengths of diffusion models and transformer architectures. \n\nDiffusion models are a class of generative models that iteratively refine the input data by adding noise and then removing it, learning the reverse process to generate new data samples. \n\nTransformer models, on the other hand, are primarily used for natural language processing and are known for their ability to handle sequential data and capture long-range dependencies.\n\nBy integrating these two concepts, diffusion transformers aim to leverage the power of diffusion models for generative tasks, such as image synthesis, while utilizing the strengths of transformer models in handling complex sequential data.\n\nThe key benefits of diffusion transformers include:\n\n1. **Improved generation quality**: By iteratively refining the input data, diffusion transformers can produce high-quality samples that rival those of state-of-the-art generative mod

In [25]:
ask(query2)

'Compared to the standard transformer architecture, the diffusion transformer architecture has several key differences:\n\n1. **Diffusion-based encoder**: The diffusion transformer uses a diffusion-based encoder, which iteratively refines the input data by adding noise and then removing it, whereas the standard transformer encoder uses self-attention mechanisms to process the input data.\n2. **Reverse process**: The diffusion transformer learns the reverse process of the diffusion model, which involves removing the noise that was added during the encoding process, whereas the standard transformer does not have a reverse process.\n3. **Sequential refinement**: The diffusion transformer refines the input data sequentially, using the output from the previous step as the input for the next step, whereas the standard transformer processes the input data in parallel using self-attention mechanisms.\n4. **Noise schedule**: The diffusion transformer uses a noise schedule to control the amount 

In [26]:
ask(query3)

"I'd be happy to provide a more detailed explanation of diffusion transformers.\n\n**Diffusion Models**\n\nDiffusion models are a class of generative models that work by iteratively refining the input data through a process of adding noise and then removing it. This process is repeated multiple times, with the model learning to reverse the diffusion process to generate new data samples.\n\nThe diffusion process consists of two main steps:\n\n1. **Forward process**: The input data is gradually corrupted by adding noise, resulting in a sequence of increasingly noisy versions of the data.\n2. **Reverse process**: The model learns to reverse the diffusion process by removing the noise, resulting in a generated sample that is similar to the original input data.\n\n**Transformer Models**\n\nTransformer models, on the other hand, are primarily used for natural language processing and are known for their ability to handle sequential data and capture long-range dependencies. They consist of an 

In [27]:
ask(query4)

"**Executive Summary**\n\nThe AI revolution in India is transforming the country's economy, society, and industries at an unprecedented pace. With a growing pool of talented engineers, a thriving startup ecosystem, and a supportive government, India is emerging as a hub for AI innovation. This report provides an in-depth analysis of the current state of AI in India, its applications, challenges, and future prospects.\n\n**Introduction**\n\nArtificial Intelligence (AI) has been gaining momentum globally, and India is no exception. The country has made significant strides in adopting AI technologies, with a growing number of startups, research institutions, and corporates investing heavily in AI research and development. The Indian government has also launched several initiatives to promote AI adoption, including the establishment of the National Artificial Intelligence Mission (NAIM) and the creation of a task force on AI.\n\n**Current State of AI in India**\n\nThe AI ecosystem in India

In [28]:
ask(query5)

"We were discussing the topic of diffusion transformers and their architecture. You had asked me to explain the concept in more detail, and I had provided a detailed explanation of how diffusion transformers work, their components, and their benefits.\n\nTo continue the conversation, I'd like to ask: What specific aspect of diffusion transformers would you like to explore further? Are you interested in learning more about their applications, their comparison to other generative models, or something else?"

In [29]:
hist_docs = memory_store.similarity_search("", k=100)
len(hist_docs)

6